In [ ]:
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.cross_validation import train_test_split, cross_val_predict
from breast_cancer_functions import pick_best_features, how_many_features_do_we_want
%matplotlib inline

In [ ]:
conn = sqlite3.connect('breast_cancer.db')
c = conn.cursor()

df = pd.read_sql('''SELECT *
                    FROM cancer''', conn)

# this gets run when I'm done working for the session
conn.close()

In [ ]:
# this is the X I will use
all_ = list(df.columns[2:])
X = df[all_]
X = X.assign(const=1)

# make the y out of the diagnosis column, this can be used for all of the dataframes
y = [1 if diag == 'M' else 0 for diag in df.diagnosis]

In [ ]:
# check features for how useful they are, check my functions file for more indepth explanation
num_features_to_check = X.shape[1]
features_ranking = pick_best_features(X, y, num_features_to_check)

In [ ]:
# see how many features I should use, check my functions file for more indepth explanation
results, features = how_many_features_do_we_want(features_ranking, X, y)

In [ ]:
for key, value in results.iteritems():
    print key, value

In the case of diagnosing breast cancer false positives are more acceptable than false negatives. If we incorrectly tell someone she has breast cancer we can test again to see if we got it wrong, she's scared for a bit but we don't send her away with cancer. If we incorrectly tell someone she doesn't have breast cancer and she trully does, she walks out thinking she is in the clear while the cancer may be getting worse, not okay.

With that said, looking at the results from testing LinearRegression models with different number of features it looks like it really stops improving at about the top 13 features. After that there is improvement but not much and I haven't even done a train test split or cross validation yet; I'm not modeling anything yet, just figureing out what features I want to use. Basically it's EDA without graphing anything.

So I am going to take the first 13 spots in features: `features[:13]`

In [ ]:
# this is the X I will be working with
X = X[features[:13]]

# split into training and testing
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [ ]:
# model with cross validation
model = LogisticRegression()
predicted = cross_val_predict(model, X_train, y_train, cv=5)

tp, tn, fp, fn = 0, 0, 0, 0
for num in zip(np.array(y_train),predicted):
    if num == (1,1):
        tp += 1
    elif num == (1,0):
        fn += 1
    elif num == (0,1):
        fp += 1
    elif num == (0,0):
        tn += 1
        
tp, tn, fp, fn

I'm going to perform grid search on every classification model I can think of and compare them tomorrow. Start using Precision and Recall.